In [23]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer

In [24]:
import warnings
warnings.filterwarnings("ignore")

In [25]:
genre_mapping = {'0ef0aa70-f86c-4141-8054-8b39af97867d': 'Biography',
                '287a1485-7a88-4c2f-bc94-ca418b6c47a1': 'Cartoons',
                '66fad8c3-d84f-458d-a8bf-5b4f154969e0': 'Show',
                '7b7c97f6-1adb-4b43-bfe8-9455812fac0b': 'Theater',
                '9fa28b61-a257-4a3e-945b-a9ef76a146d6': 'Fantasy',
                'b0836a1d-635f-4d89-bcc5-25d10ba56642': 'Anime',
                'd7214feb-8c11-4aea-aabb-ac98a8d56fd5': 'History',
                'dc65dbc8-34ba-4df1-b32c-4f895e10bff8': 'Shorts',
                'eb001d27-5be3-4d42-9d88-90d593f2627d': 'War_movies',
                '364fdc2e-bdfe-40be-b2c5-d30f43ec432e': 'Crime',
                '1f22ccf1-288a-4e6e-b39a-7502799e7125': 'Лекции',
                '6d640e04-be3a-4c8c-852e-4e9b12449d5d': 'Концерты',
                '7b7c97f6-1adb-4b43-bfe8-9455812fac0b': 'Театр',
                '2f7908cc-e2fd-43cf-b626-ec1aef436160': 'Курсы'}

In [26]:
# actor director ?

In [27]:
with open('/home/jovyan/work/item_coldstart_dataset.pkl', 'rb') as f:
    df = pickle.load(f)

In [28]:
df.columns

Index(['actor', 'age_access_type', 'country', 'average_rating', 'duration',
       'availability', 'type', 'name', 'release_year', 'genre', 'director',
       'target', 'subscription_only', 'uid'],
      dtype='object')

# Unhashing

In [29]:
all_genres = []

for item_genres in df['genre']:
    all_genres.extend(item_genres)

In [30]:
all_unique_genres = set(all_genres)

In [31]:
len(all_unique_genres)

32

In [32]:
all_unique_genres

{'0ef0aa70-f86c-4141-8054-8b39af97867d',
 '287a1485-7a88-4c2f-bc94-ca418b6c47a1',
 '364fdc2e-bdfe-40be-b2c5-d30f43ec432e',
 '3e6e08b4-2bb0-46d6-aee7-98780e394c86',
 '5743ecbe-a141-47d6-a7d7-e800f41cb6f5',
 '5c403894-146a-47a4-ae75-9f1956a30dbb',
 '66fad8c3-d84f-458d-a8bf-5b4f154969e0',
 '6d640e04-be3a-4c8c-852e-4e9b12449d5d',
 '7b7c97f6-1adb-4b43-bfe8-9455812fac0b',
 '9fa28b61-a257-4a3e-945b-a9ef76a146d6',
 'Action',
 'Adventure',
 'ArtHouse',
 'Comedy',
 'Detective',
 'Documentary',
 'Drama',
 'Family',
 'ForKids',
 'Horror',
 'Humor',
 'Melodrama',
 'Music',
 'Sci-Fi',
 'Sport',
 'Thriller',
 'Travel',
 'b0836a1d-635f-4d89-bcc5-25d10ba56642',
 'd7214feb-8c11-4aea-aabb-ac98a8d56fd5',
 'd7f6b51c-6ebe-4b9c-9aad-2c60792a7d9c',
 'dc65dbc8-34ba-4df1-b32c-4f895e10bff8',
 'eb001d27-5be3-4d42-9d88-90d593f2627d'}

In [33]:
decoded_genres = [genre_mapping.get(genre, genre) for genre in all_unique_genres]

In [34]:
# 5c403894-146a-47a4-ae75-9f1956a30dbb
# 3e6e08b4-2bb0-46d6-aee7-98780e394c86
# d7f6b51c-6ebe-4b9c-9aad-2c60792a7d9c
# 5743ecbe-a141-47d6-a7d7-e800f41cb6f5
decoded_genres

['Melodrama',
 'Thriller',
 'Drama',
 'Biography',
 'ForKids',
 'Концерты',
 'Family',
 '5743ecbe-a141-47d6-a7d7-e800f41cb6f5',
 'Театр',
 'Music',
 'Documentary',
 'Sci-Fi',
 'Comedy',
 '3e6e08b4-2bb0-46d6-aee7-98780e394c86',
 'Show',
 'Anime',
 'Crime',
 'History',
 'Detective',
 'Travel',
 'War_movies',
 'Fantasy',
 'Sport',
 'Shorts',
 '5c403894-146a-47a4-ae75-9f1956a30dbb',
 'Horror',
 'Action',
 'Humor',
 'Adventure',
 'd7f6b51c-6ebe-4b9c-9aad-2c60792a7d9c',
 'Cartoons',
 'ArtHouse']

In [35]:
all_genres = []

for col in ['actor', 'country', 'genre', 'director']:
    print(f'======{col}======')
    all_values = []
    for i, item_values in enumerate(df[col]):
        try:
            all_values.extend(item_values)
        except:
            print(i, item_values)
    all_values = set(all_values)
    print('Len:', len(all_values))
    # print(all_values)

======actor======
Len: 34028
======country======
Len: 108
======genre======
Len: 32
======director======
2408 nan
2446 nan
Len: 8191


In [36]:
df.drop([2408, 2446], axis=0, inplace=True)

In [37]:
mltpl_cat_features = ['actor', 'country', 'genre', 'director']
cat_features = ['availability']
cat_features_raw = ['type', 'subscription_only']
num_features = ['age_access_type', 'average_rating', 'duration', 'release_year']
skewed_num_features = ['duration']
text_features = ['name']
mltpl_cat_n_top = {'actor': 5, 'country': 1, 'genre': 3, 'director': 1}

class Preprocesser:
    def __init__(self, 
                mulpiple_cat_features: list,
                skewed_num_features: list,
                text_features: list,
                cat_features: list,
                mltpl_cat_n_top: dict):
        self.mulpiple_cat_features = mulpiple_cat_features
        self.skewed_num_features = skewed_num_features
        self.text_features = text_features
        self.mltpl_cat_n_top = mltpl_cat_n_top
        self.cat_features = cat_features
       # self.release_year_feature = release_year_feature
        self.embed_model = SentenceTransformer('DeepPavlov/rubert-base-cased')
        
    def __get_top_n_mltpl_cat_features(self, df_pr, col):
        all_cats = []
        for element_values in df_pr[col]:
            all_cats.extend(element_values)
        return Counter(all_cats).most_common(self.mltpl_cat_n_top[col])
            
    def __preprocess_mulpiple_cat_features(self, df_pr):
        for col in self.mulpiple_cat_features:
            feature_number = self.mltpl_cat_n_top[col]
            df_pr[[f'{col}_{i}' for i in range(feature_number)]] = pd.DataFrame(np.full((len(df), feature_number), np.nan))
            for i in df_pr.index:
                names = df_pr.loc[i, col]
                for k, name in enumerate(names[:int(feature_number)]):
                    df_pr.loc[i, f'{col}_{k}'] = name
            df_pr.drop(col, axis=1, inplace=True)

        return df_pr
    
    def __preprocess_skewed_num_features(self, df_pr):
        for col in self.skewed_num_features:
            df_pr[col] = df_pr[col].apply(lambda x: np.log1p(x))
        return df_pr
    
    def __preprocess_text_features(self, df_pr):
        for col in self.text_features:
            df_pr[[f'{col}_{i}' for i in range(768)]] = self.embed_model.encode(df_pr[col].values)
            df_pr.drop(col, axis=1, inplace=True)
        return df_pr
    
    def __get_unique_values(self, feature):
        unique_values = []
        for element in feature.dropna():
            unique_values.extend(element)
        return set(unique_values)
    
    def __preprocess_cat_features(self, df_pr):
        for col in self.cat_features:
            unique_values = self.__get_unique_values(df_pr[col])
            cats_df = pd.DataFrame(np.full((len(df_pr), len(unique_values)),
                                            0),
                                   columns=list(unique_values))
            
            for i, element in enumerate(df_pr[col]):
                if isinstance(element, float):
                    cats_df.loc[i, :] = 'Na'
                    continue
                for value in element:
                    cats_df.loc[i, value] = '1'
            df_pr.drop(col, axis=1, inplace=True)
        df_pr[cats_df.columns] = cats_df
        return df_pr  
            
    def preprocess(self,
                   df: pd.DataFrame):
        df_pr = df.copy()
        df_pr = self.__preprocess_mulpiple_cat_features(df_pr)
        df_pr = self.__preprocess_skewed_num_features(df_pr)
        df_pr = self.__preprocess_cat_features(df_pr)
        df_pr = self.__preprocess_text_features(df_pr)
        
        return df_pr
        
        
class TargetTransformer:
    def __init__(self, denominator=1):
        self.scale = denominator
        
    def inverse_transform(self, feature):
        return feature.apply(lambda x: np.expm1(x))
    
    def transform(self, feature):
        return feature.apply(lambda x: np.log1p(x))

In [39]:
preprocesser = Preprocesser(mltpl_cat_features,
            skewed_num_features,
            text_features,
            cat_features,
            mltpl_cat_n_top)

target_transformer = TargetTransformer()

No sentence-transformers model found with name /home/jovyan/.cache/torch/sentence_transformers/DeepPavlov_rubert-base-cased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/jovyan/.cache/torch/sentence_transformers/DeepPavlov_rubert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identica

In [40]:
df_modern = df[df['release_year'] >= 2000]
df_short = df_modern[df_modern['target'] < 0.02]
df_short.reset_index(drop=True, inplace=True)
df_pr = preprocesser.preprocess(df_short)

df_pr['average_rating'] = df_pr['average_rating'].astype(float)
df_pr['age_access_type'] = df_pr['age_access_type'].astype(int)
df_pr['subscription_only'] = df_pr['subscription_only'].apply(int).astype(str)

In [41]:
df_pr

,age_access_type,average_rating,duration,type,release_year,target,subscription_only,uid,actor_0,actor_1,...,name_758,name_759,name_760,name_761,name_762,name_763,name_764,name_765,name_766,name_767
0,18,4.00,15.607270,MOVIE,2019,0.000004,0,11ba66db-e941-4c3a-8da6-d8900e56f8c7,cff3362f-ad91-498d-80b3-ea3ab8ec65cf,bcad17eb-f43b-4b51-864c-d03317b392d4,...,0.033705,0.438382,0.117099,-0.449425,0.368870,0.176638,0.422600,0.022174,-0.064221,0.357092
1,18,6.99,15.145235,MOVIE,2011,0.000354,1,3f30a2ef-53b7-40e3-954f-1bdfc38a6d17,cff3362f-ad91-498d-80b3-ea3ab8ec65cf,30dd19ca-115c-4d1a-a508-74174e866829,...,-0.155257,0.573361,0.033664,-0.075815,0.193481,-0.420364,0.116773,-0.090162,0.096753,-0.353996
2,18,6.00,15.145235,MOVIE,2006,0.000064,1,cdfa700f-122d-41e5-b8dc-9c6813bab6d2,cff3362f-ad91-498d-80b3-ea3ab8ec65cf,a805fdd3-7415-42f0-9d00-b1a9f1a670ac,...,-0.171907,0.299762,-0.329399,-0.084129,0.296856,-0.134781,0.188332,0.126907,0.401704,-0.318638
3,0,6.21,15.396549,MOVIE,2006,0.000003,1,d8b55146-dee5-4498-957c-5614414b48fb,cff3362f-ad91-498d-80b3-ea3ab8ec65cf,a51b3766-26a9-4064-8774-5b3b0ff3f38c,...,0.150038,0.777897,-0.351048,-0.236852,0.139988,0.039575,0.110862,-0.258445,-0.264087,0.132789
4,6,6.40,15.523889,MOVIE,2022,0.009705,1,ba6bec1a-3aa9-48c7-aa65-908c21627a12,cff3362f-ad91-498d-80b3-ea3ab8ec65cf,873fd466-9242-4413-b0f6-23812f622629,...,0.142848,1.254483,-0.551046,-0.098492,0.185326,0.191771,0.265626,-0.014733,0.038026,-0.180334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9478,16,7.60,18.121735,SERIAL,2015,0.000085,0,f4b8a993-5d91-41f4-b5fb-3769a73d4b53,d47abe6d-d856-40cc-877b-395072f26804,c2cdf6a3-81e4-4fb1-89d4-2f55fa60eef3,...,0.029255,0.251564,-0.221529,-0.274407,0.172291,-0.095824,0.306483,-0.155895,-0.250882,0.050630
9479,18,NaN,17.184185,SERIAL,2022,0.000696,1,673a98a8-5d12-4a3d-9c50-d1d196a8fd96,d3ac2bba-c855-40b1-98ed-5a360484846c,03968c90-4be3-46a4-96cd-a14735d689d4,...,-0.063956,0.535674,-0.443978,0.167658,0.522164,0.223132,0.864783,0.706656,0.564934,-0.213584
9480,16,7.80,17.627492,SERIAL,2020,0.000652,0,37305c51-cb1e-4ff2-b5ad-3425ed73b17a,40977851-22fb-4829-8c01-d73acaa39ffe,17cf0f03-562b-4891-b93e-b3d186ff1fc9,...,0.168566,0.453653,0.045424,0.112483,0.348138,0.071285,0.691507,0.388921,0.101442,0.205210
9481,16,NaN,16.358686,SERIAL,2020,0.001266,1,249b3db3-8dbd-4a18-bd4b-5d0cc08878c4,8f485b8e-48f3-4d18-80fa-6fbaa7838b10,66fdabb2-0aea-49e5-9fb3-7f6f73ba96cf,...,-0.357440,0.430215,-0.289068,-0.122204,0.410426,-0.053278,0.010176,-0.185435,-0.070059,-0.052059


In [42]:
with open('dataset_1.pkl', 'wb') as f:
    pickle.dump(df_pr, f)